In [59]:
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from src.utils.kpt_utils import get_active_kpts, get_image_patches, kpts_2_img_coordinates

In [48]:
sample = torch.load('/home/yannik/vssil/eval/ulosd_manipulator_128pix/ulosd_sample_9/sample.pt')
kpts = torch.load('/home/yannik/vssil/eval/ulosd_manipulator_128pix/ulosd_sample_9/key_points.pt')
patches = torch.load('/home/yannik/vssil/eval/ulosd_manipulator_128pix/ulosd_sample_9/patches.pt')
N, T, C, H, W = sample.shape
print(sample.shape)
print(kpts.shape)
print(patches.shape)

torch.Size([1, 30, 3, 128, 128])
torch.Size([1, 30, 64, 3])
torch.Size([1, 30, 64, 3, 12, 12])


In [49]:
active_kpts = get_active_kpts(kpts, 0.5)
# Adapt
active_kpts[..., :2] *= -1 
K, D = active_kpts.shape[-2:]
active_img_coordinates = kpts_2_img_coordinates(active_kpts, (H, W))
print(active_kpts.shape)

torch.Size([1, 30, 4, 3])


In [50]:
active_kpt_patches = get_image_patches(sample, active_kpts, (15, 15))
Hp, Wp = active_kpt_patches.shape[-2:]
print(active_kpt_patches.shape)

torch.Size([1, 30, 4, 3, 15, 15])


In [51]:
import matplotlib.patches as pts

for t in range(T):
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.imshow(sample[0, t].permute(1, 2, 0).cpu().numpy())
    for k in range(K):
        kpt_w = active_img_coordinates[0, t, k, 0]
        kpt_h = active_img_coordinates[0, t, k, 1]
        ax.scatter(kpt_w, kpt_h, color='lime')
        rect = pts.Rectangle((kpt_w - int(.5*Wp), kpt_h - int(.5*Hp)), Wp, Hp,
                             linewidth=1, linestyle='--', edgecolor='lime', facecolor='none')
        ax.add_patch(rect)
    ax.axis('off')
    fig.savefig(f'/home/yannik/save_png/frame_{t}.png', bbox_inches='tight')
    plt.close()

In [95]:
from kornia.filters import Sobel

sobel = Sobel()

for t in range(1):
    for k in range(2, 3):
        
        grad = sobel(active_kpt_patches[0:1, t, k])
        
        red = torch.zeros_like(grad)
        red[0, 0] += (grad[0, 0] * 5)
        green = torch.zeros_like(grad)
        green[0, 1] += (grad[0, 1] * 5)
        blue = torch.zeros_like(grad)
        blue[0, 2] += (grad[0, 2] * 5)
        
        plt.figure()
        plt.imshow(active_kpt_patches[0, t, k].permute(1, 2, 0).cpu().numpy())
        plt.axis('off')
        # ptl.savefig('')
        plt.close()
        
        cn = ['red', 'green', 'blue']
        for i, c in enumerate([red, green, blue]):
            fig = plt.figure()
            plt.imshow(c[0].permute(1, 2, 0).cpu().numpy())
            plt.axis('off')
            #plt.show()
            fig.savefig(f'/home/yannik/save_png/{cn[i]}.png', bbox_inches='tight')
            plt.close()
        

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [64]:
font = {'family' : 'Sans',
        'weight' : 'bold',
        'size'   : 12}

matplotlib.rc('font', **font)

In [99]:
bins = 10

for t in range(1):
    for k in range(K):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        colors = ['red', 'green', 'blue']
        for c, ch in zip(colors, range(3)):
            hist = torch.histc(active_kpt_patches[0, t, k, ch], bins=bins)/(Hp*Wp)
            ax.bar(np.arange(0, bins), hist, zs=ch, color=c, zdir='y')
            ax.set_yticks([])
            #ax.set_zticks(np.arange(0, 1.1, 0.2))
            ax.set_zticks([])
            #ax.set_xticks(np.arange(0, bins))
            ax.set_xticks([])
        #plt.show()
        fig.savefig(f'/home/yannik/save_png/color_hist_{t}_{k}.png', bbox_inches='tight')
        plt.close()